In [3]:

import pandas as pd
import numpy as np

#image tools
from matplotlib import pyplot as plt
from PIL import Image

#filesystem tools that allow for file manipulation
import os
from glob import glob

#ML Stuff



import tensorflow as tf

%matplotlib inline

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:

graph = tf.Graph()
with graph.as_default():

  # Variables.
  batch_size = 128  #mini batch for SGD
  beta = .001  #regularization 
  image_size = 28
  num_labels = 10


  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(mnist.validation.images)
  tf_test_dataset = tf.constant(mnist.test.images)

  # weights and biases for output/logit layer
  w_logit = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  b_logit = tf.Variable(tf.zeros([num_labels]))

  


  def model(data):
        """
        Assembles the NN
        """
        return tf.matmul(data, w_logit) + b_logit  #return the output layer


  # Training Computations
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  regularized_loss = tf.nn.l2_loss(w_logit)
  total_loss = loss + beta * regularized_loss

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(total_loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [7]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
 
    # Generate a minibatch.
    batch_data, batch_labels = mnist.train.next_batch(batch_size)


    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), mnist.validation.labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), mnist.test.labels))

Initialized
Minibatch loss at step 0: 11.780111
Minibatch accuracy: 9.4%
Validation accuracy: 11.9%
Minibatch loss at step 500: 0.587761
Minibatch accuracy: 89.8%
Validation accuracy: 87.2%
Minibatch loss at step 1000: 0.379731
Minibatch accuracy: 87.5%
Validation accuracy: 90.1%
Minibatch loss at step 1500: 0.164555
Minibatch accuracy: 95.3%
Validation accuracy: 91.4%
Minibatch loss at step 2000: 0.461937
Minibatch accuracy: 91.4%
Validation accuracy: 91.6%
Minibatch loss at step 2500: 0.293370
Minibatch accuracy: 92.2%
Validation accuracy: 91.8%
Minibatch loss at step 3000: 0.122560
Minibatch accuracy: 97.7%
Validation accuracy: 92.3%
Minibatch loss at step 3500: 0.291566
Minibatch accuracy: 89.8%
Validation accuracy: 92.3%
Minibatch loss at step 4000: 0.286241
Minibatch accuracy: 94.5%
Validation accuracy: 92.2%
Minibatch loss at step 4500: 0.350204
Minibatch accuracy: 89.8%
Validation accuracy: 92.0%
Minibatch loss at step 5000: 0.199112
Minibatch accuracy: 93.0%
Validation accurac

In [ ]:
with tf.Session(graph=graph) as session:
    valid_prediction.eval()

In [ ]:
mnist.validation.images